In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import glob


def plot_all():
    file_names = glob.glob("logs/*")
    for file_name in file_names:
        variables = re.search(r"logs\\(.*?)_(.*?)_(.*?)_(\d*?)(_.*)?.log", file_name)
        if variables is None:
            continue
        kp = float(variables.group(1))
        ki = float(variables.group(2))
        kd = float(variables.group(3))
        alt = int(variables.group(4))
        info = variables.group(5)[1:] if variables.group(5) is not None else None
        with open(file_name) as f:
            log = f.read()
            plot(log, alt, kp, ki, kd, info)


def plot(log: str, alt, kp, ki, kd, info: str | None):
    # Parse log data
    lines = log.strip().split("\n")
    data = [line for line in lines if not line.startswith(">") and not line.startswith("time")]
    data = [list(map(float, row.split(','))) for row in data]
    data = np.array(data)
    events = [line.split() for line in lines if line.split()[0] == ">"]

    # Extract time, throttle, and expected landing altitude
    times = data[:, 0]
    throttles = np.clip(data[:, 1], 0, 1)
    expected_landing_altitudes = data[:, 2]

    # Create the plot
    ax1: plt.Axes
    ax2: plt.Axes
    fig1, [ax1, ax2] = plt.subplots(ncols=2, figsize=(16, 6))

    # Plot throttle
    ax1.set_xlabel("Time")
    ax1.set_ylabel("Throttle")
    ax1.set_ylim(top=1.1, auto=True)
    ax1.axhline(y=1, color="lightgray")
    ax1.axhline(y=0.8, color="gray", linestyle="--")
    ax1.plot(times, throttles, color="blue")

    # Plot expected landing altitude
    ax2.set_xlabel("Time")
    ax2.set_ylabel("Expected Land Altitude")
    #ax2.set_ylim(bottom=-10, top=10)
    ax2.axhline(y=0, color="gray", linestyle="--")
    ax2.plot(times, expected_landing_altitudes, color="red")

    # Plot events as vertical lines on both plots
    for event in events:
        event_type = event[1]
        event_time = float(event[2])
        color = "red" if event_type == "START" else "green"
        ax1.axvline(x=event_time, color=color, alpha=0.5)
        ax2.axvline(x=event_time, color=color, alpha=0.5)

    # Add a title
    fig1.suptitle(
        f"PID Controller (Alt={alt}, Kp={kp}, Ki={ki}, Kd={kd}"
        + (f", {info})" if info is not None else ")")
    )

    # Display the plot
    plt.show()


plot_all()